# Single image super-resolution demonstration

Select GPU hardware acceleration (TPU not supported by TensorFlow 2) in Runtime > Change runtime type

In [0]:
%tensorflow_version 2.x

from google.colab import drive
drive.mount('/content/gdrive')

## Repository clone and dataset download (first time only)

In [0]:
"""
%mkdir gdrive/My Drive/Part_II_Dissertation
%cd gdrive/My Drive/Part_II_Dissertation
!git clone https://github.com/EmptyJackson/video-super-resolution
"""

In [0]:
"""
%cd video-super-resolution
"""

## Repository update (subsequent uses)

In [0]:
%cd gdrive/My Drive/Part_II_Dissertation/video-super-resolution

In [0]:
!git pull

## Demonstration

In [0]:
#!python src/tools/crop_image.py data/div2k/DIV2K_valid_LR_bicubic/X2/0855x2.png lr_crop.png 240
#!python src/tools/crop_image.py data/div2k/DIV2K_valid_HR/0855.png hr_orig.png 240 standard 2
!python src/tools/crop_image.py data/div2k/DIV2K_train_LR_bicubic/X4/0791x4.png lr_crop.png 240
!python src/tools/crop_image.py data/div2k/DIV2K_train_HR/0791.png hr_orig.png 240 standard 4
#!python src/tools/crop_image.py data/div2k/DIV2K_train_LR_bicubic/X2/0049x2.png lr_crop.png 240
#!python src/tools/crop_image.py data/div2k/DIV2K_train_HR/0049.png hr_orig.png 240 standard 2
!python src/upscale_image.py lr_crop.png hr_pred.png network checkpoints/edsr_x4_240p/ 140
!python src/upscale_image.py lr_crop.png hr_bicb.png bicubic 4

"""
!python src/tools/crop_image.py lr_crop.png lr_crop.png 240 square
!python src/tools/crop_image.py hr_bicub.png hr_bicb.png 480 square
!python src/tools/crop_image.py hr_pred.png hr_pred.png 480 square
"""

In [0]:
import matplotlib.pyplot as plt
import numpy as np

s = 4
x = np.array([200, 300])
y = np.array([50, 150])
adj = True

def crop(im, x, y):
  return im[y[0]:y[1], x[0]:x[1], :]

fig = plt.figure(figsize=(15,15))
fig.add_subplot(2, 2, 1)
im = plt.imread('lr_crop.png')
if adj:
  im = crop(im, x, y)
plt.axis('off')
plt.imshow(im)
fig.add_subplot(2, 2, 2)
im = plt.imread('hr_bicb.png')
if adj:
  im = crop(im, x*s, y*s)
plt.axis('off')
plt.imshow(im)
fig.add_subplot(2, 2, 3)
im = plt.imread('hr_orig.png')
if adj:
  im = crop(im, x*s, y*s)
plt.axis('off')
plt.imshow(im)
fig.add_subplot(2, 2, 4)
im = plt.imread('hr_pred.png')
if adj:
  im = crop(im, x*s, y*s)
plt.axis('off')
plt.imshow(im)
#plt.subplots_adjust(wspace=0.05)
plt.show()